<a href="https://colab.research.google.com/github/Petinelli-Inc/ajustePressaoSIS4/blob/main/pressaoSIS4_MERV13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from eppy.modeleditor import IDF
from eppy.results import fasthtml

import pprint

In [22]:
iddFile = "/content/Energy+.idd"
idfFile = "/content/BS_IMPEX_ADM_R00.idf"
newTable = "/content/eplustbl.htm"
table = "/content/BS_IMPEX_ADM_R00Table.html"
epwFile = "/content/BRA_SC_Florianopolis-Luz.Intl.AP.838990_TMYx.2003-2017.epw"

localPrefix = "/content"

In [ ]:
IDF.setiddname(iddFile)
idf = IDF(idfFile, epwFile)
pp = pprint.PrettyPrinter()

In [ ]:
pszhp = idf.idfobjects['HVACTEMPLATE:SYSTEM:UNITARYHEATPUMP:AIRTOAIR']
qtdepszhp = len(pszhp)

powerFansArray = []
pressureFansArray = []
fanMechEfficiency = 0.7

In [8]:
# Função para ajustar pressão conformo a nova FanPower definida
def ajustePRESSAO(tableName):
    filehandle = open(tableName, 'r')
    fansOutput = fasthtml.tablebyname(filehandle, "Fans")
    fansOutputTable = fansOutput[1]
    global powerFansArray, pressureFansArray

    filehandle = open(table, 'r')
    zoneCooling = fasthtml.tablebyname(filehandle, "Zone Sensible Cooling")
    zoneCoolingTable = zoneCooling[1]

    for n in range(qtdepszhp):
        maxAirFlow = fansOutputTable[n+1][4]
        maxAirFlowCFM = round(maxAirFlow * 2118.88, 2) 
        filterMERV13 = round((zoneCoolingTable[13] * 21188.88) * 0.9, 5)
        bhp = (maxAirFlowCFM * 0.00094) + filterMERV13     

        if bhp <= 1.00:
            fanMotorEfficiency = 0.825
        if 1.00 < bhp <= 2.00 :
            fanMotorEfficiency = 0.84
        if 2.00 < bhp <= 5.00:
            fanMotorEfficiency = 0.875
        if 5.00 < bhp <= 10:
            fanMotorEfficiency = 0.895
        if 10 < bhp <= 20:
            fanMotorEfficiency = 0.91
        if 20 < bhp <= 30:
            fanMotorEfficiency = 0.924
        if 30 < bhp <= 50:
            fanMotorEfficiency = 0.93
        if 50 < bhp <= 60:
            fanMotorEfficiency = 0.936
        if 60 < bhp <= 75:
            fanMotorEfficiency = 0.941
        if 75 < bhp <= 125:
            fanMotorEfficiency = 0.945
        if 125 < bhp <= 200:
            fanMotorEfficiency = 0.95
        
        powerfanWATT = round(bhp * (746 / fanMotorEfficiency), 8)
        newPressure = round(powerfanWATT * fanMechEfficiency / (maxAirFlow), 8)
        powerFansArray.append(powerfanWATT)
        pressureFansArray.append(newPressure)
        pszhp[n].Supply_Fan_Delta_Pressure = newPressure
        pszhp[n].Supply_Fan_Motor_Efficiency = fanMotorEfficiency

        print("System Name: " + fansOutputTable[n+1][0])
        print("Adjusted Pressure: " + str(newPressure))
    idf.save()
    

In [13]:
def segundoAjuste(tableName):
    filehandle = open(tableName, 'r')
    fansOutput = fasthtml.tablebyname(filehandle, "Fans")
    fansOutputTable = fansOutput[1]

    global powerFansArray, pressureFansArray, pszhp
    
    print("New values from ajusted table:")
    for n in range(qtdepszhp):
        newSecPressure = round((powerFansArray[n] * pressureFansArray[n]) / fansOutputTable[n+1][5], 8)

        print("System Name: " + fansOutputTable[n+1][0])
        print("New Pressure: " + str(newSecPressure))

        pszhp[n].Supply_Fan_Delta_Pressure = newSecPressure
    idf.saveas(localPrefix + "\pressureAdjusted.idf")


In [ ]:
ajustePRESSAO(table)
idf.run(output_directory='D:\DADOS\Desktop\Python\idf\SIS4')
segundoAjuste(newTable)
